## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-24-23-49-58 +0000,nypost,Nvidia expands AI empire with Groq licensing ...,https://nypost.com/2025/12/24/business/nvidia-...
1,2025-12-24-23-44-50 +0000,nyt,"Honduras Declares Nasry Asfura, Trump Ally, Wi...",https://www.nytimes.com/2025/12/24/world/ameri...
2,2025-12-24-23-39-24 +0000,bbc,Democratic lawmaker sues to remove Trump's nam...,https://www.bbc.com/news/articles/c79x0x7v70go...
3,2025-12-24-23-31-42 +0000,missionlocal,"UCSF tells workers at Ward 86, site of fatal s...",https://missionlocal.org/2025/12/ucsf-ward-86-...
4,2025-12-24-23-31-00 +0000,wsj,China Developing Options to Threaten Taiwan Wi...,https://www.wsj.com/world/china/china-developi...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2383/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
15,trump,21
83,christmas,21
277,new,17
89,epstein,10
168,ukraine,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
173,2025-12-24-10-30-00 +0000,wsj,Capital Account: President Trump’s social-medi...,https://www.wsj.com/economy/central-banking/wh...,67
226,2025-12-24-01-08-43 +0000,bbc,"Trump trips, a fake video and 10 possible co-c...",https://www.bbc.com/news/articles/c5yj4yvx80do...,67
216,2025-12-24-02-09-44 +0000,nypost,Trump administration to send 350 National Guar...,https://nypost.com/2025/12/23/us-news/trump-ad...,61
88,2025-12-24-17-22-59 +0000,nypost,DOJ seeks Christmas-week volunteers to speed u...,https://nypost.com/2025/12/24/us-news/doj-plea...,55
115,2025-12-24-15-15-38 +0000,nypost,Ukraine agrees to demilitarized economic zone ...,https://nypost.com/2025/12/24/us-news/ukraine-...,54


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
173,67,2025-12-24-10-30-00 +0000,wsj,Capital Account: President Trump’s social-medi...,https://www.wsj.com/economy/central-banking/wh...
88,55,2025-12-24-17-22-59 +0000,nypost,DOJ seeks Christmas-week volunteers to speed u...,https://nypost.com/2025/12/24/us-news/doj-plea...
40,51,2025-12-24-21-10-21 +0000,nypost,New details revealed in gruesome NYC garbage t...,https://nypost.com/2025/12/24/us-news/new-deta...
102,41,2025-12-24-16-15-06 +0000,nyt,Zelensky Open to Pulling Back Troops in Easter...,https://www.nytimes.com/2025/12/24/world/europ...
143,37,2025-12-24-12-26-16 +0000,nypost,Explosion in Moscow kills three people — inclu...,https://nypost.com/2025/12/24/world-news/explo...
92,34,2025-12-24-16-53-16 +0000,nypost,Afghan DC National Guard shooting suspect Rahm...,https://nypost.com/2025/12/24/us-news/afghan-d...
9,32,2025-12-24-23-12-53 +0000,wapo,Trump-backed conservative candidate wins Hondu...,https://www.washingtonpost.com/world/2025/12/2...
66,32,2025-12-24-19-21-49 +0000,nypost,California law could force 2M homeowners to ri...,https://nypost.com/2025/12/24/us-news/californ...
10,28,2025-12-24-23-07-16 +0000,nypost,Passenger explodes at Alaska Airlines staff fo...,https://nypost.com/2025/12/24/us-news/passenge...
68,27,2025-12-24-19-07-18 +0000,nypost,Italian leader Giorgia Meloni gives year-end a...,https://nypost.com/2025/12/24/world-news/itali...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
